# Goal: Create Interactive Map of 2025 NCAA Participants

In [31]:
### Setup and Dependencies

### Import constants from config.py
from config import *

### Dependencies
import os
import geopandas as gpd
import pandas as pd
import numpy as np
import seaborn as sns
import sqlite3  # Assuming SQL connection for database operations

### MATPLOTLIB WITH ACCESORIES
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import matplotlib.image as mpimg
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from matplotlib.legend_handler import HandlerTuple
from matplotlib.legend_handler import HandlerBase
from PIL import Image
## Map visualization
import folium
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap
from folium.features import CustomIcon


### FILE PATHS
TEMP_FOLDER = '../TEMP/'
DATA_FOLDER = '../data/'

## 2024-25 Full Roster Path
roster_path = DATA_FOLDER + 'roster_2025_current_march_25_v4_ex20250325.csv'

full_roster = pd.read_csv(roster_path) # load roster as dataframe
full_roster.info() # Check to make sure it loaded correctly



### SCHOOL INFO TABLE FOR LOGO PATHS
school_info_path = os.path.join('..', 'data', 'arena_school_info.csv')
school_info_df = pd.read_csv(school_info_path) # Load school info

# Path to logo folder
logo_folder = os.path.join('..', 'images', 'logos')

### SHAPEFILES
# Path to .geojson file with State Boundries
geojson_path = os.path.join('..', 'data', 'vault', 'combined-us-canada.geojson')
# Load the states shapefile
gdf_states = gpd.read_file(geojson_path)

# Path to shapefile with all US counties
shapefile_path = os.path.join('..', 'data', 'vault', 'cb_2018_us_county_500k.shp')
gdf = gpd.read_file(shapefile_path)
# Set the initial CRS (assuming it's in EPSG:4326, but you may need to verify the original CRS)
gdf = gdf.set_crs(epsg=4326)

## CHECK SHAPEFILES FOR COMPATIBILITY
# Set the CRS for both dataframes if it's missing
if gdf.crs is None:
    gdf.set_crs(epsg=4326, inplace=True)  # Assuming coordinates are in WGS 84 (lat/lon)

if gdf_states.crs is None:
    gdf_states.set_crs(epsg=4326, inplace=True)  # Assuming coordinates are in WGS 84 (lat/lon)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1836 entries, 0 to 1835
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Current Team    1836 non-null   object 
 1   Last_Name       1836 non-null   object 
 2   First_Name      1836 non-null   object 
 3   No              1836 non-null   int64  
 4   Position        1836 non-null   object 
 5   Yr              1836 non-null   object 
 6   Ht              1836 non-null   object 
 7   Wt              1836 non-null   int64  
 8   DOB             1835 non-null   object 
 9   Hometown        1835 non-null   object 
 10  Height_Inches   1836 non-null   int64  
 11  Draft_Year      228 non-null    float64
 12  NHL_Team        228 non-null    object 
 13  D_Round         228 non-null    float64
 14  Last Team       1828 non-null   object 
 15  League          1781 non-null   object 
 16  City            1835 non-null   object 
 17  State_Province  1835 non-null   o

#### Roster Hotfix
- replace missed location abbreviations with full names
    - RUS = Russia
    - IN = S_P 'Indiana' + Country 'USA'
    - UKR = Ukraine
    - JPN = Japan

In [32]:
## Location abbreviation replacement dictionary
loc_replace_dict = {'RUS':'Russia', 'IN':'Indiana', 'UKR':'Ukraine', 'JPN':'Japan'}

# Check the State_Province column and replace the abbreviations with full names
full_roster['State_Province'] = full_roster['State_Province'].replace(loc_replace_dict)
# same with the Country column
full_roster['Country'] = full_roster['Country'].replace(loc_replace_dict)

# If Country = Indiana, change to USA
full_roster['Country'] = full_roster['Country'].replace('Indiana', 'USA')

In [33]:
### Filter Roster to only teams in the NCAA Tournament
print(ncaa_team_list_2025) # Check the list of Tourney teams from the config file

# Filter to only teams in the NCAA Tournament
roster_ncaa = full_roster[full_roster['Current Team'].isin(ncaa_team_list_2025)]

roster_ncaa.rename(columns={'Current Team':'Team'}, inplace=True) # Rename 'Current_Team' to 'Team' for consistency

# Create a new 'Player' column that combines the player's first and last name
roster_ncaa['Player'] = roster_ncaa['First_Name'] + ' ' + roster_ncaa['Last_Name']

roster_ncaa['Player'] = roster_ncaa['Player'].str.strip() # Strip any leading or trailing white space

# roster_ncaa.info() # Check to make sure it loaded correctly

['Michigan State', 'Cornell', 'Boston University', 'Ohio State', 'Western Michigan', 'Minnesota State', 'Minnesota', 'Massachusetts', 'Boston College', 'Bentley', 'Providence', 'Denver', 'Maine', 'Penn State', 'Connecticut', 'Quinnipiac']


C:\Users\jbanc\AppData\Local\Temp\ipykernel_21944\2994776235.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  roster_ncaa.rename(columns={'Current Team':'Team'}, inplace=True) # Rename 'Current_Team' to 'Team' for consistency
C:\Users\jbanc\AppData\Local\Temp\ipykernel_21944\2994776235.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  roster_ncaa['Player'] = roster_ncaa['First_Name'] + ' ' + roster_ncaa['Last_Name']
C:\Users\jbanc\AppData\Local\Temp\ipykernel_21944\2994776235.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

## Add Current Season Stats to Roster

In [34]:
### Load the player_ytd stats table from the database

# connect to the database
conn = sqlite3.connect(recent_clean_db)
# Check the table name in db
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())


# Load the player_ytd stats table from the database
player_ytd = pd.read_sql_query("SELECT * FROM player_stats_ytd", conn)
player_ytd.rename(columns={'Clean_Player':'Player'}, inplace=True) # Rename 'Clean Player' to 'Player' for consistency
player_ytd['Player'] = player_ytd['Player'].str.strip() # Strip leading and trailing whitespace from the 'Player' column
# player_ytd.info() # Check to make sure it loaded correctly

# Close the connection
conn.close()

### Merge the player_ytd stats table with the roster_ncaa table on Player and Team columns
# roster_ncaa_ytd = pd.merge(roster_ncaa, player_ytd, on=['Player', 'Team'], how='left')
roster_ncaa_ytd = pd.merge(roster_ncaa, player_ytd, on='Player', how='left')

roster_ncaa_ytd.rename(columns={'Team_x':'Team'}, inplace=True)# Rename Team_x back to Team
roster_ncaa_ytd.rename(columns={'Team_y':'Team_from_db'}, inplace=True) # Rename Team_y to Team_from_db

# roster_ncaa_ytd.info() # Check to make sure it merged correctly
# roster_ncaa_ytd.head() # Head of the merged table

[('player_stats_ytd',), ('master_roster',), ('advanced_metrics',), ('game_details',), ('goalie_stats',), ('line_chart',), ('linescore',), ('penalty_summary',), ('player_stats',), ('scoring_summary',)]


#### Filter out Skaters who have not appeared in a game this season

In [35]:
## How many players show no games played?
# no_games = roster_ncaa_ytd[roster_ncaa_ytd['Games_Played'].isnull()]
# print(len(no_games))
# print(no_games)



In [36]:
## Drop any players who have not appeared in a game this season
## Print pre filter length
print(f'PreFiltered Roster Length: {len(roster_ncaa_ytd)}')
roster_ncaa_ytd = roster_ncaa_ytd.dropna(subset=['Games_Played'])
print(f'PostFiltered Roster Length: {len(roster_ncaa_ytd)}')

### Value count by Position
# print(roster_ncaa_ytd['Position'].value_counts())

PreFiltered Roster Length: 443
PostFiltered Roster Length: 413


In [37]:
# Group by the location columns and count the number of players in each group
location_summary = roster_ncaa_ytd.groupby(['City', 'State_Province', 'Country']).size().reset_index(name='Player_Count')

# Display the resulting DataFrame
print(location_summary)

               City    State_Province   Country  Player_Count
0        Abbotsford  British Columbia    Canada             1
1          Aberdeen          Scotland  Scotland             1
2          Abington     Massachusetts       USA             1
3           Airdrie           Alberta    Canada             1
4        Albert Lea         Minnesota       USA             1
..              ...               ...       ...           ...
309        Woodbury         Minnesota       USA             3
310  Woodcliff Lake        New Jersey       USA             1
311       Woodhaven          Michigan       USA             1
312       Yaroslavl            Russia    Russia             1
313          Zilina          Slovakia  Slovakia             1

[314 rows x 4 columns]


In [38]:
## Value COunts of the location data (City, State, Country)
# print(len(roster_ncaa_ytd['City'].value_counts()))
# roster_ncaa_ytd['City'].value_counts()
# roster_ncaa_ytd['State_Province'].value_counts()
# roster_ncaa_ytd['Country'].value_counts()

## Get Geocode Locations for all hometowns in the dataset
- Using Google Maps API

In [39]:
# ### GEOCODING USING GOOGLE MAPS API 
# # LCHECK FOR AND LOAD GEOCODED DATA BEFORE RUNNING - THIS COSTS MONEY

# import googlemaps
# import pandas as pd
# import config




# # Initialize the Google Places API client
# gmaps = googlemaps.Client(key=config.g_key)

# def geocode_google_places(row):
#     try:
#         location_str = f"{row['City']}, {row['State_Province']}, {row['Country']}"
#         print(f"Querying location: {location_str}")  # Debugging output
#         geocode_result = gmaps.geocode(location_str)
        
#         # Check the API response
#         print(f"Geocode result: {geocode_result}")  # Debugging output
        
#         # Check if we got a valid result
#         if geocode_result and 'geometry' in geocode_result[0]:
#             location = geocode_result[0]['geometry']['location']
#             return pd.Series([location['lat'], location['lng']])
#         else:
#             return pd.Series([None, None])  # Return None if no valid result
#     except Exception as e:
#         print(f"Error encountered: {e}")  # Debugging output
#         return pd.Series([None, None])  # Handle errors gracefully

# # Apply the geocode function to the data using Google Places API
# location_summary[['Latitude', 'Longitude']] = location_summary.apply(geocode_google_places, axis=1)

# # Filter out rows with missing coordinates if needed
# location_summary_transformed = location_summary.dropna(subset=['Latitude', 'Longitude'])

# # Display the cleaned data with coordinates
# location_summary_transformed.head()

In [40]:
## Save the geocoded data to a CSV file to avoid re-running the API
# location_summary_transformed.to_csv(DATA_FOLDER + '2025_tourney_location_summary_geocoded.csv', index=False)

In [41]:
## Load the geocoded data from the CSV file
location_summary_geocoded = pd.read_csv(DATA_FOLDER + '2025_tourney_location_summary_geocoded.csv')
location_summary_geocoded.head()

,City,State_Province,Country,Player_Count,Latitude,Longitude
0,Abbotsford,British Columbia,Canada,1,49.050438,-122.304470
1,Aberdeen,Scotland,Scotland,1,57.149889,-2.093753
2,Abington,Massachusetts,USA,1,42.104823,-70.945322
3,Airdrie,Alberta,Canada,1,51.292697,-114.013411
4,Albert Lea,Minnesota,USA,1,43.647801,-93.368656


In [42]:
### MERGE THE LATITUDE AND LONGITUDE DATA WITH THE ROSTER DATA
roster_ncaa_ytd_location = pd.merge(roster_ncaa_ytd, location_summary_geocoded, on=['City', 'State_Province', 'Country'], how='left')

# Check the merged data
print(roster_ncaa_ytd_location.info())
roster_ncaa_ytd_location.head()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 413 entries, 0 to 412
Data columns (total 36 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Team            413 non-null    object 
 1   Last_Name       413 non-null    object 
 2   First_Name      413 non-null    object 
 3   No              413 non-null    int64  
 4   Position        413 non-null    object 
 5   Yr              413 non-null    object 
 6   Ht              413 non-null    object 
 7   Wt              413 non-null    int64  
 8   DOB             413 non-null    object 
 9   Hometown        413 non-null    object 
 10  Height_Inches   413 non-null    int64  
 11  Draft_Year      114 non-null    float64
 12  NHL_Team        114 non-null    object 
 13  D_Round         114 non-null    float64
 14  Last Team       412 non-null    object 
 15  League          403 non-null    object 
 16  City            413 non-null    object 
 17  State_Province  413 non-null    obj

,Team,Last_Name,First_Name,No,Position,Yr,Ht,Wt,DOB,Hometown,...,TOI_sec,PIM,FOW,FOL,Games_Played,FO%,TOI,Player_Count,Latitude,Longitude
0,Connecticut,Carabin,Nick,27,Defensemen,Gr,5-10,195,4/30/2000,"Mahwah, N.J.",...,41423.0,16.0,0.0,0.0,37.0,NaN,11:30:23,1,41.079210,-74.184170
1,Connecticut,Gustafsson Nyberg,Viking,6,Defensemen,So,6-6,205,9/21/2003,"Stockholm, Sweden",...,40821.0,27.0,0.0,0.0,37.0,NaN,11:20:21,6,59.332704,18.065625
2,Connecticut,Janviriya,Kai,11,Defensemen,Fr,5-8,170,4/2/2005,"Bloomfield, Mich.",...,43621.0,6.0,0.0,0.0,37.0,NaN,12:07:01,1,42.581333,-83.281177
3,Connecticut,Messineo,Tom,15,Defensemen,Jr,6-0,190,5/2/2002,"Westwood, Mass.",...,42881.0,20.0,0.0,0.0,37.0,NaN,11:54:41,2,42.213987,-71.224499
4,Connecticut,Pascucci,Jack,2,Defensemen,Jr,6-2,195,2/15/2002,"North Andover, Mass.",...,4295.0,4.0,0.0,0.0,10.0,NaN,01:11:35,1,42.698702,-71.135058


## Create the Map Using Follium

### Circular Offset helper Function

In [43]:
import math

# Function to apply a circular offset to markers with the same location
def add_circular_offset(lat, lon, count, index, radius=0.007):
    """
    Distributes markers in a circular pattern around a central point.
    The radius increases slightly with the number of markers to prevent overlap.
    """
    # Calculate angle in radians (360 degrees divided by number of markers)
    angle = (360 / count) * index
    radians = math.radians(angle)

    # Dynamic adjustment of the radius: the more markers, the larger the radius
    dynamic_radius = radius * (1 + (count / 5))  # Scale the radius based on the number of markers

    # Offset latitude and longitude using circular placement
    lat_offset = lat + (dynamic_radius * math.cos(radians))  # Offset based on cosine
    lon_offset = lon + (dynamic_radius * math.sin(radians))  # Offset based on sine

    return lat_offset, lon_offset


In [ ]:
######### RENAME THE DATA TO MERGED_DF
merged_df = roster_ncaa_ytd_location

# Assign unique index per player in each city group
merged_df['city_group_index'] = merged_df.groupby(['City', 'State_Province', 'Country']).cumcount()

# Assign 'Player_Count' per city directly to 'merged_df' using 'transform'
merged_df['Player_Count'] = merged_df.groupby(['City', 'State_Province', 'Country'])['First_Name'].transform('count')

# Set Logo Size (tuple of width and height in pixels)
logo_size = (55, 55)  # Adjust as needed

# Convert all number columns to int
int_columns = ['No', 'Height_Inches', 'Wt', 'Draft_Year', 'D_Round', 
               'G', 'A', 'Pts', 'plus_minus', 'Sh', 'PIM', 'Games_Played']

for col in int_columns:
    merged_df[col] = merged_df[col].astype('Int64')

import math

def create_map_with_team_logos(merged_df, school_info_df, logo_folder, gdf_states, map_center=[45.0, -93.0], zoom_start=4):
    # Initialize the map
    folium_map = folium.Map(location=map_center, zoom_start=zoom_start, tiles='OpenStreetMap', name='Default Map')

    # ---- ADD BASE LAYERS ----
    # Add additional base layers (you can add more as needed)
    # folium.TileLayer('OpenStreetMap', name='Default Map').add_to(folium_map)
    # folium.TileLayer('Stamen Terrain', name='Terrain', attr=".").add_to(folium_map)
    # folium.TileLayer('Stamen Toner', name='Toner', attr=".").add_to(folium_map)
    folium.TileLayer('CartoDB dark_matter', name='Dark Theme', attr=".").add_to(folium_map)
    folium.TileLayer('CartoDB positron', name='Light Theme', attr=".").add_to(folium_map)
    

    # ---- ADD CHOROPLETH LAYER ----
    # Create 'state_counts_df' from 'merged_df'
    state_counts = merged_df['State_Province'].value_counts()
    state_counts_df = pd.DataFrame(state_counts).reset_index()
    state_counts_df.columns = ['State_Province', 'Player_Count']

    # Create Custom Bins for Choropleth to better control look
    # Define custom bins to handle the wide distribution
    custom_bins = [0, 1, 5, 10, 20, 50, 100, 200, 250] # Adjust as needed

    # Convert the GeoDataFrame to GeoJSON using __geo_interface__
    geojson_data = gdf_states.__geo_interface__

    ### ORIGINAL CODE ###
    # Add the Choropleth directly to the map with a name for LayerControl
    folium.Choropleth(
        geo_data=geojson_data,
        data=state_counts_df,
        columns=['State_Province', 'Player_Count'],
        key_on='feature.properties.name',  # Adjust this if necessary
        fill_color='YlGn',
        fill_opacity=0.5,
        line_opacity=0.2,
        legend_name='Number of Players by State/Province',
        bins=custom_bins,  # Apply custom bins
        reset=True,  # Ensure the choropleth is reset based on new bins
        name='Shade by Player Count'
    ).add_to(folium_map)

    ############## NEW CODE ##############
    # ---- ADD STATE LABELS LAYER ----
    # Merge 'state_counts_df' with 'gdf_states' to get centroids
    gdf_states_subset = gdf_states[['name', 'geometry']]  # Adjust 'name' if your GeoDataFrame has a different column name
    state_counts_gdf = gdf_states_subset.merge(state_counts_df, left_on='name', right_on='State_Province')

    # Calculate centroids
    state_counts_gdf['centroid'] = state_counts_gdf.geometry.centroid

    # Create a FeatureGroup for the labels
    labels_layer = folium.FeatureGroup(name='Players Count by State')

    # Add labels to the labels_layer
    for idx, row in state_counts_gdf.iterrows():
        # Get centroid coordinates
        lat = row['centroid'].y
        lon = row['centroid'].x
        # Get player count
        player_count = row['Player_Count']
        # Create a text label
        label = folium.Marker(
            location=[lat, lon],
            icon=folium.DivIcon(
                html=f'''
                    <div style="
                        font-family: Optima, sans-serif;
                        font-weight: bold;
                        font-size: 16px;
                        color: black;
                        text-align: center;
                        
                        padding: 2px;
                        
                    ">
                        {player_count}
                    </div>
                '''
            )
        )
        labels_layer.add_child(label)

    # Add the labels_layer to the map
    labels_layer.add_to(folium_map)


########### NOT BIG FAN OF COLOR SCHEME
    # # ---- ADD HEATMAP LAYER ----
    # # Create heat_data from merged_df
    # heat_data = [[row['Latitude'], row['Longitude']] for idx, row in merged_df.iterrows()]

    # # Create a FeatureGroup for the heatmap layer
    # heatmap_layer = folium.FeatureGroup(name='Heatmap')

    # # Define a custom gradient for better color transitions
    # custom_gradient = {
    #     0.2: '#ADD8E6',  # Light Blue for low intensity
    #     0.4: '#00FF00',  # Green for mid-low intensity
    #     0.6: '#FFFF00',  # Yellow for mid-high intensity
    #     0.8: '#FFA500',  # Orange for high intensity
    #     1.0: '#FF0000'   # Red for maximum intensity
    # }

    # # Add the HeatMap to the FeatureGroup with adjusted parameters
    # HeatMap(
    #     heat_data, 
    #     radius=15,                # Increase radius for smoother heat blobs
    #     blur=15,                  # Slightly increase blur to smooth transitions
    #     max_intensity=100,         # Adjust max intensity for better scaling
    #     gradient=custom_gradient, # Use the custom gradient
    #     min_opacity=0.4           # Slight opacity for low intensity
    # ).add_to(heatmap_layer)

    # # Add the heatmap layer to the map
    # heatmap_layer.add_to(folium_map)
    
    
    # ---- ADD HEATMAP LAYER ----
    # Create heat_data from merged_df
    heat_data = [[row['Latitude'], row['Longitude']] for idx, row in merged_df.iterrows()]

    # Create a FeatureGroup for the heatmap layer
    heatmap_layer = folium.FeatureGroup(name='Heatmap')

    # Add the HeatMap to the FeatureGroup
    HeatMap(heat_data, radius=25, blur=15, max_intensity=20).add_to(heatmap_layer)

    # Add the heatmap layer to the map
    heatmap_layer.add_to(folium_map)

    # ---- MARKER CLUSTER LAYER ----
    cluster_group = folium.FeatureGroup(name='Individual Players', control=True, show=False)
    marker_cluster = MarkerCluster(
        spiderfy_on_max_zoom=True,
        show_coverage_on_hover=False,
        max_cluster_radius=20,
        disableClusteringAtZoom=14,
        animateAddingMarkers=True,
        zoomToBoundsOnClick=True
    ).add_to(cluster_group)

    # Compute the mean latitude and longitude for centering the map
    Latitude = merged_df['Latitude'].mean()
    Longitude = merged_df['Longitude'].mean()

    # Create the map centered on the computed mean Latitude and Longitude
    map_instance = folium.Map(location=[Latitude, Longitude], zoom_start=12)

    # Add the cluster group to the map but initially hidden
    map_instance.add_child(cluster_group)

    # Define a custom script to toggle the visibility of the cluster group on zoom
    map_instance.get_root().html.add_child(folium.Element(f'''
        <script>
            var clusterLayer = {cluster_group.get_name()};
            var map = {map_instance.get_name()};
            map.on('zoomend', function() {{
                if (map.getZoom() >= 14) {{
                    if (!map.hasLayer(clusterLayer)) {{
                        map.addLayer(clusterLayer);
                    }}
                }} else {{
                    if (map.hasLayer(clusterLayer)) {{
                        map.removeLayer(clusterLayer);
                    }}
                }}
            }});
        </script>
    '''))


    # Loop through the merged_df to place markers
    for idx, row in merged_df.iterrows():
        # Retrieve team and logo information
        team_name = row['Team']
        logo_info = school_info_df[school_info_df['Team'] == team_name]['logo_abv'].values

        if len(logo_info) > 0:
            logo_abv = logo_info[0]
            logo_path = os.path.join(logo_folder, f"{logo_abv}.png")

            if os.path.exists(logo_path):
                logo_icon = CustomIcon(logo_path, icon_size=logo_size)

                player_count = row['Player_Count']
                current_index = row['city_group_index']

                # Apply circular offset for overlapping markers
                if player_count > 1:
                    lat_offset, lon_offset = add_circular_offset(
                        row['Latitude'], row['Longitude'], player_count, current_index
                    )
                else:
                    lat_offset, lon_offset = row['Latitude'], row['Longitude']  # No offset if only one player

                # Enhance the tooltip with player information, including hometown
                tooltip_html = f"""
                <div style="font-size: 14px; font-family: Arial;">
                    <strong>{row['First_Name']} {row['Last_Name']} - {row['Team']}</strong><br>
                    {row['Hometown']}<br>
                    {row['Yr']} {row['Position']}<br>
                    {f"<div style='font-size: 12px; color: gray; margin-top: 5px;'>SEASON STATS:<br> {row['Games_Played']} GP, {row['G']} G, {row['A']} A, {row['Pts']} PTS, {row['PIM']} PIM</div>" if pd.notna(row['Games_Played']) else ""}
                </div>
                """


                # Add player marker with the custom logo icon and enhanced tooltip
                folium.Marker(
                    location=[lat_offset, lon_offset],
                    tooltip=folium.Tooltip(tooltip_html),
                    icon=logo_icon
                ).add_to(marker_cluster)

    # Add the marker cluster layer to the map
    cluster_group.add_to(folium_map)

    # ---- ADD LAYER CONTROL ----
            
    folium.LayerControl().add_to(folium_map)

    # Inject custom CSS for styling the LayerControl
    custom_css = """
    <style>
    /* Style for the Layer Control List */
    .leaflet-control-layers-list {
        font-size: 18px;  /* Increase font size */
        line-height: 1.5; /* Ensure adequate spacing between lines */
    }

    /* Style for the checkboxes and radio buttons */
    .leaflet-control-layers input[type="radio"], 
    .leaflet-control-layers input[type="checkbox"] {
        transform: scale(1.5);  /* Scale the size of the checkbox/radio button */
        margin-right: 8px;      /* Add space between the button and label */
    }

    /* Optional: Style the background of the layer control to make it stand out */
    .leaflet-control-layers {
        background-color: white;  /* Ensure the control has a visible background */
        border-radius: 5px;       /* Slight rounding of the control edges */
        padding: 10x;
        box-shadow: 0px 0px 5px rgba(0,0,0,0.3);  /* Add a shadow for better visibility */
    }
    </style>
    """

    # Add the custom CSS to the map's HTML
    folium_map.get_root().html.add_child(folium.Element(custom_css))

    # Return the map after processing all markers
    return folium_map

# Assuming 'gdf_states' is already defined in your code
enhanced_player_map = create_map_with_team_logos(merged_df, school_info_df, logo_folder, gdf_states)

# Save the map to an HTML file for visualization
enhanced_map_file_path = os.path.join('..', 'TEMP', 'MAP', 'player_origin_map_with_stats_v1.html')
enhanced_player_map.save(enhanced_map_file_path)


C:\Users\jbanc\AppData\Local\Temp\ipykernel_21944\4217382445.py:71: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  state_counts_gdf['centroid'] = state_counts_gdf.geometry.centroid
